# Trade-Offs in Missing Data Conventions
None of these approaches is without trade-offs: use of a separate mask array requires allocation of an additional Boolean array, which adds overhead in both storage and computation. A sentinel value reduces the range of valid values that can be represented, and may require extra (often non-optimized) logic in CPU and GPU arithmetic. Common special values like NaN are not available for all data types.

# Missing Data in Pandas

Keep in mind that NaN is specifically a floating-point value; there is no equivalent NaN value for integers, strings, or other types.

|Typeclass|Conversion When Storing NAs|NA Sentinel Value|
|:---:|:---:|:---:|
|floating|No change|np.nan|
|object|No change|None or np.nan|
|integer|Cast to float64|np.nan|
|boolean|Cast to object|None or np.nan|

Keep in mind that in Pandas, string data is always stored with an object dtype.

# Operating on Null Values

## Detecting null values

In [3]:
import pandas as pd
import numpy as np

data = pd.Series([1, np.nan, 'hello', None])
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [4]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [5]:
data[data.notnull()]

0        1
2    hello
dtype: object

## Dropping null values

In [6]:
data.dropna()

0        1
2    hello
dtype: object

In [7]:
df = pd.DataFrame([[1, np.nan, 2],
                  [2, 3, 5],
                  [np.nan, 4, 6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [8]:
df.dropna()

,0,1,2
1,2.0,3.0,5


In [9]:
df.dropna(axis='columns')

,2
0,2
1,5
2,6


In [11]:
df.dropna(axis=1)

,2
0,2
1,5
2,6


In [12]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [13]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [14]:
df.dropna(axis='columns', how='any')

,2
0,2
1,5
2,6


In [15]:
df.dropna(axis='rows', thresh=3)  # having 3 non-null values at least

,0,1,2,3
1,2.0,3.0,5,NaN


## Filling null values

In [16]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [17]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [19]:
# forward-fill
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [20]:
# back-fill
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

In [21]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [22]:
df.fillna(method='ffill', axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0
